In [17]:
import torch
import torchvision
import torchvision.transforms as transforms

Define functions for data augmentation of training set.
If we encounter RGB( 3 channels ) images , we will use the following to normalize:
```
transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
```


In [18]:
transform_train = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
])

Load the CIFAR-10 Dataset by downloading it with torchvision's datasets function.

torch.utils.data.DataLoader class represents a Python iterable over a dataset.

In [30]:
opt_batch = 16
trainset = torchvision.datasets.CIFAR10(root='.', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=opt_batch, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='.', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=opt_batch, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


Define a cnn model here. (I copied the parameters directly from other's model.)

In [20]:
import torch.nn as nn

class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()

    self.conv_layer = nn.Sequential(
        #block 1
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),

        #block 2
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout2d(p=0.05),

        #block 3
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.fc_layer = nn.Sequential(
        nn.Dropout(p=0.1),
        nn.Linear(4096, 1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.1),
        nn.Linear(512, 10)
    )
  
  def forward(self, x):
    x = self.conv_layer(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x


net = CNN()

Define a loss function using cross-entropy loss (for classification) and Adam optimizer.

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Train & Test the neural network.

In [ ]:
from torch.autograd import Variable

def calculate_accuracy(loader):
  correct = 0
  total = 0
  accur = 0.0
  with torch.no_grad():
    for data in loader:
      images, labels = data
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  accur = correct / total * 100
  return accur

epoch_size = 50

for epoch in range(epoch_size):
  running_loss = 0.0
  for i,data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = Variable(inputs), Variable(labels)
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()

    if epoch > 16:
      for group in optimizer.param_groups:
        for p in group['params']:
          state = optimizer.state[p]
          if state['step'] >= 1024:
            state['step'] = 1000
    optimizer.step()
    running_loss += loss.item()
  
  running_loss /= len(trainloader)
  train_accur = calculate_accuracy(trainloader)
  test_accur = calculate_accuracy(testloader)

  print("Iteration: {0} | Loss: {1} | Training accuracy: {2}% | Test accuracy: {3}%".format(epoch+1, running_loss, train_accur, test_accur))

print('==> Finished Training ...')

Iteration: 1 | Loss: 1.0925072313404083 | Training accuracy: 67.794% | Test accuracy: 66.16%
Iteration: 2 | Loss: 0.9023744855165482 | Training accuracy: 70.666% | Test accuracy: 70.82000000000001%
Iteration: 3 | Loss: 0.8017339448261261 | Training accuracy: 75.486% | Test accuracy: 73.11%
Iteration: 4 | Loss: 0.7297243852877617 | Training accuracy: 76.58% | Test accuracy: 74.15%
Iteration: 5 | Loss: 0.6839256379318237 | Training accuracy: 78.018% | Test accuracy: 77.38000000000001%
Iteration: 6 | Loss: 0.6390557837605476 | Training accuracy: 80.502% | Test accuracy: 77.97%
Iteration: 7 | Loss: 0.6065029391860962 | Training accuracy: 82.33200000000001% | Test accuracy: 79.03999999999999%
Iteration: 8 | Loss: 0.5654581157708168 | Training accuracy: 81.91199999999999% | Test accuracy: 78.35%
Iteration: 9 | Loss: 0.5463207822176814 | Training accuracy: 82.974% | Test accuracy: 79.79%
Iteration: 10 | Loss: 0.5242690891569853 | Training accuracy: 83.908% | Test accuracy: 79.61%
Iteration: 1